In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('samsung_price.xlsx')

In [3]:
stock_list = pd.read_html('https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]

In [4]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.03.23,70500,200,70600,71200,70300,12332242
1,2022.03.22,70300,400,69900,70500,69900,9402666
2,2022.03.21,69900,800,70900,71000,69900,11169002
3,2022.03.18,70700,500,70600,70900,70200,14410038
4,2022.03.17,71200,800,71200,71800,70900,17646315


In [5]:
stock_list['종목코드'] = stock_list['종목코드'].map('{:06d}'.format)
stock_list.sort_values(by='종목코드', inplace=True)
stock_list.reset_index(drop=True, inplace=True)
stock_list.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,동화약품,000020,의약품 제조업,"의약품 제조,판매,수출입",1976-03-24,12월,유준하,http://www.dong-wha.co.kr,서울특별시
1,KR모터스,000040,그외 기타 운송장비 제조업,"이륜차(오토바이) 제조,도매",1976-05-25,12월,노성석,http://www.krmotors.com,경상남도
2,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
3,메리츠화재,000060,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
4,삼양홀딩스,000070,기타 금융업,"지주회사,경영자문컨설팅",1968-12-27,12월,"엄태웅, 윤재엽",http://www.samyang.com,서울특별시


In [6]:
code_list = stock_list['종목코드'][:10]

In [7]:
# code_list = ['323410']

In [8]:
from bs4 import BeautifulSoup
import requests

In [9]:
def get_df(code_list):
    df = pd.DataFrame()
    for i in range(len(code_list)):
        print(f'{len(code_list)} 번째 중 {i+1} 번째 인 {code_list[i]} 조회 중..')
        code = code_list[i]
        url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
        req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        html = BeautifulSoup(req.text, "lxml")
        pgrr = html.find('td', class_='pgRR')
        s = pgrr.a['href'].split('=')
        last_page = s[-1]
        for page in range(1, int(last_page)+1):
            req = requests.get(f'{url}&page={page}', headers={'User-agent': 'Mozilla/5.0'})
            df = pd.concat([df, pd.read_html(req.text, encoding = 'euc-kr')[0]], ignore_index=True)
            df.dropna(inplace=True)
            # df = df.loc[::-1]
            df.reset_index(drop=True, inplace=True)
        
        # 전일대비 종가 등락률 계산
        close = df['종가']
        daily_rate = (close / close.shift(-1)-1)* 100
        df['일간변동률(%)'] = daily_rate
        
        # 전일대비 거래량 등락률 계산
        volume = df['거래량']
        volume_rate = (volume / volume.shift(-1)-1) * 100
        df['거래량증감률(%)'] = volume_rate
        
    return df

In [10]:
test = get_df(code_list)

10 번째 중 1 번째 인 000020 조회 중..
10 번째 중 2 번째 인 000040 조회 중..
10 번째 중 3 번째 인 000050 조회 중..
10 번째 중 4 번째 인 000060 조회 중..
10 번째 중 5 번째 인 000070 조회 중..
10 번째 중 6 번째 인 000080 조회 중..
10 번째 중 7 번째 인 000100 조회 중..
10 번째 중 8 번째 인 000120 조회 중..
10 번째 중 9 번째 인 000140 조회 중..
10 번째 중 10 번째 인 000150 조회 중..


In [11]:
# 데이터가 없는 행 일괄 삭제
# test.dropna(inplace = True)
# 인덱스 재배열
# test.reset_index(drop=True, inplace=True)

In [12]:
test

,날짜,종가,전일비,시가,고가,저가,거래량,일간변동률(%),거래량증감률(%)
0,2022.03.25,13600.0,0.0,13700.0,13700.0,13500.0,46129.0,0.000000,-72.015724
1,2022.03.24,13600.0,50.0,13600.0,13700.0,13500.0,164839.0,-0.366300,-58.389751
2,2022.03.23,13650.0,0.0,13800.0,14100.0,13600.0,396150.0,0.000000,-10.213434
3,2022.03.22,13650.0,200.0,13950.0,14200.0,13550.0,441213.0,-1.444043,-16.655868
4,2022.03.21,13850.0,500.0,14450.0,14550.0,13800.0,529387.0,-3.484321,-80.665363
...,...,...,...,...,...,...,...,...,...
6429,1996.07.10,35400.0,600.0,34400.0,35800.0,34400.0,53280.0,1.724138,39.075959
6430,1996.07.09,34800.0,500.0,34700.0,35800.0,34700.0,38310.0,-1.416431,-67.264804
6431,1996.07.08,35300.0,600.0,35100.0,36700.0,34200.0,117030.0,1.729107,78.046554
6432,1996.07.06,34700.0,200.0,34000.0,35900.0,34000.0,65730.0,0.579710,-62.345325
